In [ ]:
import sys
sys.path.append('../..')
import numpy as np
import matplotlib.pyplot as plt
import math
from exploration.config import mongo_inst

In [ ]:
osu_db = mongo_inst['osu_random_db']

In [ ]:
love = osu_db['beatmap_criteria_curve'].find_one({'_id': 104229})

In [ ]:
plt.plot(np.asarray(love['no_mod']['n_pass'])/np.asarray(love['no_mod']['total']))

In [ ]:
bm_popularity_pipeline = [
    {
        '$group': {
            '_id': '$beatmap_id',
            'count': {
                '$sum': 1
            }
        }
    },
    {
        '$sort': {
            'count': -1
        }
    }
]

In [ ]:
top_bm = list(osu_db['osu_scores_high'].aggregate(bm_popularity_pipeline))

In [ ]:
for i, bm in enumerate(top_bm):
    if bm['_id'] == 131891:
        print(i)

In [ ]:
bins = [[] for i in range(7)]
cnt = 0
for bm in top_bm:
    bm_info = osu_db['osu_beatmaps'].find_one({'_id': bm['_id']})
    if bm_info['difficultyrating'] < 1:
        continue

    b_i = min(math.floor(bm_info['difficultyrating']) - 1, 6)

    if len(bins[b_i]) < 5:
        bins[b_i].append([bm, bm_info['difficultyrating']])
        cnt += 1
    if cnt == 35:
        break

In [ ]:
for bin in bins:
    bin.sort(key = lambda x: x[1])

In [ ]:
for i, bin in enumerate(bins):
    print(f"\nMost popular {i + 1}* - {i + 2}* beatmaps:")
    for bm, _ in bin:
        compute_bm(bm)

In [ ]:
def compute_acc(score):
    c50, c100, c300, cmiss = score['count50'], score['count100'], score['count300'], score['countmiss']
    return (c50 / 6 + c100 / 3 + c300) / (c50 + c100 + c300 + cmiss)

In [ ]:
compute_bm({'_id': 2596018})

In [ ]:
def compute_bm(bm):
    bm_id = bm['_id']

    bm_scores = list(osu_db['osu_scores_high'].find({'beatmap_id': bm_id, 'enabled_mods': 0}, {'count50': 1, 'count100': 1, 'count300': 1, 'countmiss': 1, 'mlpp.est_user_pp': 1}))
    bm = osu_db['osu_beatmaps'].find_one({'_id': bm_id})

    acc_scores = np.array([[s['mlpp']['est_user_pp'], compute_acc(s)] for s in bm_scores])
    est_pp, acc = acc_scores.T
    est_pp.sort()

    eval = est_pp[len(acc_scores) - np.sum(acc > .9)]

    diff, name = bm['difficultyrating'], bm['filename']

    print(f'({bm_id}) {diff}* {name} - {eval}')

In [ ]:
MIN_ACC, PP_RANGE, PP_INTERVAL, MAX_PP = .9, 200, 50, 10000
INTERVALS = math.ceil((MAX_PP - PP_RANGE) / PP_INTERVAL)

In [ ]:
acc_scores = np.array([[s['mlpp']['est_user_pp'], compute_acc(s)] for s in bm_scores])
# est_pp, acc = acc_scores[np.argsort(acc_scores[:, 0])].T
est_pp, acc = acc_scores.T
acc = acc > MIN_ACC

In [ ]:
np.sort(est_pp)[len(acc) - np.sum(acc)]

In [ ]:
total, n_pass = np.zeros(INTERVALS), np.zeros(INTERVALS)
j_s, j_e = 0, 0

for i in range(INTERVALS):
    b = i * PP_INTERVAL

    while j_s < len(est_pp) - 1 and est_pp[j_s] < b:
        j_s+=1
    
    while j_e < len(est_pp) - 1 and est_pp[j_e] < b + PP_RANGE:
        j_e+=1
    
    total[i] = j_e - j_s
    n_pass[i] = acc[j_e] - acc[j_s]

In [ ]:
total, n_pass = np.zeros(INTERVALS), np.zeros(INTERVALS)

for i in range(INTERVALS):
    b = i * PP_INTERVAL

    for j in range(len(est_pp)):
        if b < est_pp[j] < b + PP_RANGE:
            total[i] += 1
            n_pass[i] += acc[j]

In [ ]:
x = np.arange(0, INTERVALS) * PP_INTERVAL
plt.plot(x, n_pass / total)